# Assignment 4: CNN

## Description

Implement a Convolutional Neural Network (CNN) classifier to predict whether a given icon image is the real / fake. Where the fake images were generated by TAs with a neural network.

- You are not required to use Colab in this assignment, but you have to **submit your source code**.

## Dataset

- https://lab.djosix.com/icons.zip
- 64x64 RGB jpg images


```
real/           (10000 images)
    0000.jpg
    0001.jpg
    ...
    9999.jpg
fake/           (10000 images)
    0000.jpg
    0001.jpg
    ...
    9999.jpg
unknown/        (5350 images, testing set)
    0000.jpg
    0001.jpg
    ...
    5349.jpg
```

- Training set
  - 20000 icons in `real/` and `fake/`
  - You should predict 1 for icons in `real/` and 0 for icons in `fake/`
- Testing set:
  - 5350 icons in `unknown/`
  - Your score depends on the **accuracy** on this testing set,  
    so the prediction of each icon in `unknown/` should be submitted (totally 5350 predictions, see below).


## Submission

Please upload **2 files** to E3. (`XXXXXXX` is your student ID)

1. **`XXXXXXX_4_result.json`**  
  This file contains your model prediction for the testing set.  
  You must generate this file with the function called `save_predictions()`.
2. **`XXXXXXX_4_source.zip`**  
  Zip your source code into this archive.


## Hints

- **Deep Learning Libraries**: You can use any deep learning frameworks (PyTorch, TensorFlow, ...).
- **How to implement**: There are many CNN examples for beginners on the internet, e.g. official websites of the above libraries, play with them and their model architectures to abtain high accuracy on testing set.
- **GPU/TPU**: Colab provides free TPU/GPU for training speedup, please refer to [this page in `pytut.pdf` on E3](https://i.imgur.com/VsrUh7I.png).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Include this in your code to generate result file

In [3]:
import keras  
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense ,Dropout
from keras.models import Sequential
from keras import models
from keras.optimizers import Adagrad,RMSprop, SGD, Adam, Nadam



In [4]:

import os,glob,random
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/hw4.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

## 取得資料集路徑
base_dir = '/tmp'

print(os.listdir('/tmp/'))
train_path = os.path.join(base_dir, 'train')
test_path = os.path.join(base_dir, 'test/unknown')

img_list1 = glob.glob(os.path.join(train_path, '*/*.jpg'))

print(len(img_list1))

img_list2 = glob.glob(os.path.join(test_path, '*/*.jpg'))

print(len(img_list2))



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
['tmp6bxyifvb', 'drivefs_ipc.0', 'dap_multiplexer.fddfc81dafa5.root.log.INFO.20210625-060537.51', 'initgoogle_syslog_dir.0', 'train', 'drivefs_ipc.0_shell', 'dap_multiplexer.INFO', 'debugger_11b7zpux0m', 'test']
20000
0


In [5]:
import tensorflow as tf
from keras import layers

from keras.models import Model
from keras import backend as K


# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0     

In [6]:
'''
這裡用binary_crossentropy損失來訓練模型，因為這個範例是一個二元分類問題，而最終激活函數是sigmoid。 而優化器部份使用rmsprop自動學習速率調速，學習速率初始為0.001。 在訓練期間，需監控分類的準確性。

補充說明一下，使用RMSprop優化算法優於隨機梯度下降（SGD），因為RMSprop為我們實現了自動學習速率調整
'''


from keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary',
    subset='training'
)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = val_datagen.flow_from_directory(
    train_path,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary',
    shuffle=False,
    subset='validation'
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 14000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
{0: 'fake', 1: 'real'}


In [8]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=300,  
      epochs=30,
      validation_data=validation_generator,
      validation_steps=150,
      verbose=2)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


300/300 - 50s - loss: 0.7606 - acc: 0.5657 - val_loss: 0.6241 - val_acc: 0.8117
Epoch 2/30
300/300 - 6s - loss: 0.6586 - acc: 0.6305 - val_loss: 0.1674 - val_acc: 0.9823
Epoch 3/30
300/300 - 6s - loss: 0.6172 - acc: 0.6652 - val_loss: 0.7365 - val_acc: 0.4280
Epoch 4/30
300/300 - 6s - loss: 0.5943 - acc: 0.6828 - val_loss: 0.6095 - val_acc: 0.6620
Epoch 5/30
300/300 - 6s - loss: 0.5564 - acc: 0.7205 - val_loss: 0.6294 - val_acc: 0.6863
Epoch 6/30
300/300 - 6s - loss: 0.5545 - acc: 0.7233 - val_loss: 0.5377 - val_acc: 0.7667
Epoch 7/30
300/300 - 6s - loss: 0.5371 - acc: 0.7333 - val_loss: 0.4492 - val_acc: 0.8103
Epoch 8/30
300/300 - 6s - loss: 0.5296 - acc: 0.7390 - val_loss: 0.5068 - val_acc: 0.8057
Epoch 9/30
300/300 - 6s - loss: 0.4974 - acc: 0.7560 - val_loss: 0.6204 - val_acc: 0.7193
Epoch 10/30
300/300 - 6s - loss: 0.4836 - acc: 0.7720 - val_loss: 0.5777 - val_acc: 0.7400
Epoch 11/30
300/300 - 6s - loss: 0.4527 - acc: 0.7895 - val_loss: 0.2979 - val_acc: 0.9153
Epoch 12/30
300/30

In [20]:
from keras.preprocessing.image import img_to_array, load_img

data=[]

file=os.listdir(test_path)
file.sort()


for i in file:
  if i.endswith('.jpg'):
    #load img
    image = load_img(test_path+'/'+i,target_size=(150,150))
    #change color(0~255) to (0~1)
    image = img_to_array(image).astype('float32')/255
    data.append(image)

data=np.array(data)


In [17]:
import json

def save_predictions(student_id, predictions):
  # Please use this function to generate 'XXXXXXX_4_result.json'
  # `predictions` is a list of int (0 or 1; fake=0 and real=1)
  # For example, `predictions[0]` is the prediction given "unknown/0000.jpg".
  # it will be 1 if your model think it is real, else 0 (fake).

  assert isinstance(student_id, str)
  assert isinstance(predictions, list)
  assert len(predictions) == 5350

  for y in predictions:
    assert y in (0, 1)

  with open('{}_4_result.json'.format(student_id), 'w') as f:
    json.dump(predictions, f)


In [19]:
predicts = model.predict(data)
predictions = [round(predicts[i][0]) for i in range(len(predicts))]
save_predictions('0816080', predictions)